%matplotlib inline

In [2]:
import string
import sys, os
import numpy as np
import fileinput
import subprocess
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

In [ ]:
def smooth(x, window_len=10, window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal. The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized in the begining and end part of the output signal.
    The Bartlett window is very similar to a triangular window  
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.
    output:
        the smoothed signal
    example:
    import numpy as np    
    t = np.linspace(-2,2,0.1)
    x = np.sin(t)+np.random.randn(len(t))*0.1
    y = smooth(x)
    
    see also: 
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
    """
    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."
    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."
    if window_len < 3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
    s=np.r_[2*x[0]-x[window_len:1:-1], x, 2*x[-1]-x[-1:-window_len:-1]]
    #print(len(s))
    
    if window == 'flat': #moving average
        w = np.ones(window_len,'d')
    else:
        w = getattr(np, window)(window_len)
    y = np.convolve(w/w.sum(), s, mode='same')
    return y[window_len-1:-window_len+1]

In [ ]:
def smoothTriangle(data,degree,dropVals=False):
        """performs moving triangle smoothing with a variable degree."""
        """note that if dropVals is False, output length will be identical
        to input length, but with copies of data at the flanking regions"""
        triangle=numpy.array(range(degree)+[degree]+range(degree)[::-1])+1
        smoothed=[]
        for i in range(degree,len(data)-degree*2):
                point=data[i:i+len(triangle)]*triangle
                smoothed.append(sum(point)/sum(triangle))
        if dropVals: return smoothed
        smoothed=[smoothed[0]]*(degree+degree/2)+smoothed
        while len(smoothed)<len(data):smoothed.append(smoothed[-1])
        return smoothed

In [ ]:
vars = [ 'pr' ]
scens = [ 'historical' ]
#scens = [ 'historical', 'rcp45', 'rcp85' ]
gcms = [ 'obs' ]
#gcms =  [ 'gfdl-esm2g','canesm2','cnrm-cm5','csiro-mk3-6-0','inmcm4','miroc5','mpi-esm-lr','mri-cgcm3','ccsm4',
#         'noresm1-m','bcc-csm1-1','ipsl-cm5a-lr','ipsl-cm5a-mr','gfdl-cm3','gfdl-esm2m' ]
#gcms =  [ 'gfdl-esm2g' ]
homedir=os.path.normpath("X:/projects/nicaragua/msd")
indir=os.path.normpath("X:/zraid_data5/msd/timeseries_sel")
outdir=os.path.normpath("X:/projects/nicaragua/msd/timeseries_results_jul_aug")
nyears = 30
window_lenth = 31

list="%s\%s" % (homedir,'central_america_lon_lat_elev_selected.csv')

In [ ]:
ids=[]
lons=[]
lats=[]
colnames=['Date']

In [ ]:
try:
    os.stat(outdir)
except:
    os.mkdir(outdir)

for line in fileinput.input([list]):
    if not fileinput.isfirstline():
        x=line.split(",")
        ids.append(int(x[0]))
        colnames.append(str(x[0]))
        lons.append(x[1])
        lats.append(x[2])

In [ ]:
#generate column names for file - date followed by one column per cell
colnames=range(len(ids))
n=['Date']
colnames[:0] = n

In [ ]:
for scen in scens:
    if scen == 'historical':
           start_years = [ 1970 ]
    else:
           start_years = [ 2040, 2070 ]
    for v in vars:
        for gcm in gcms:
            if gcm == 'obs':
                fname = indir + "/" + gcm+ "_" + v + "_ts.txt"
            else:
                fname = indir + "/" + gcm+ "_" + scen + "_" + v + "_ts.txt"
            fnamegz = fname + ".gz"
            if os.path.isfile(fnamegz):
                print 'Unzipping' + fnamegz
                proc=subprocess.Popen(['gzip', '-d',fnamegz])
                proc.wait()
            print 'Reading ' + fname 
            x=pd.read_csv(fname,delim_whitespace=True,header=None,names=colnames,index_col='Date',infer_datetime_format=True,parse_dates='Date')
            print 'Finished reading, zipping infile'
            p1=subprocess.Popen(['gzip',fname])
            p1.wait()

            for start_yr in start_years:
                ey=start_yr+nyears-1
                print "processing period " + str(start_yr) + "-" + str(ey)
                #make empty data frame with one row per grid cell-results for years will be appended as columns
                onset_max=pd.DataFrame(index=range(x.shape[1]))
                onset_day=pd.DataFrame(index=range(x.shape[1]))
                end_max=pd.DataFrame(index=range(x.shape[1]))
                end_day=pd.DataFrame(index=range(x.shape[1]))
                val_min=pd.DataFrame(index=range(x.shape[1]))
                duration=pd.DataFrame(index=range(x.shape[1]))
                avg_max=pd.DataFrame(index=range(x.shape[1]))
                intensity=pd.DataFrame(index=range(x.shape[1]))
                msd_stats=pd.DataFrame(index=range(x.shape[1]))
                msd_stats.index_name='N'

                for yr in range(start_yr,ey+1):
                    subx = x[datetime(yr, 5, 1):datetime(yr,9, 30)]
                    subxplus = x[datetime(yr, 4, 1):datetime(yr,10, 31)]
                    if subx[0].count() < 30:
                        continue
                    #subx_smooth=subx.apply(smooth,args=(window_len=window_lenth,window='bartlett'))
                    subx_smooth=pd.DataFrame(index=subx.index)
                    #can't figure out how to use apply with this...
                    for col in range(subx.shape[1]):
                        dat_sm=smooth(np.array(subxplus[col]),window_len=window_lenth,window='bartlett')
                        dat_smplus=pd.DataFrame(dat_sm,index=subxplus.index)
                        dat_smooth=dat_smplus[datetime(yr, 5, 1):datetime(yr,9, 30)]
                        subx_smooth = pd.concat([subx_smooth,dat_smooth],axis=1,join_axes=[subx.index])
                    subx_smooth.columns = subx.columns
                    #subx_smooth now has ndays rows x ncells columns for the current year, indexed by date
                    #Calculate statistics for current year - all columns at once
                    onset_max1yr=subx_smooth[0:int(subx.shape[0]/2)].apply(np.max)
                    onset_date1yr=subx_smooth[0:int(subx.shape[0]/2)].apply(lambda x: x.idxmax())
                    end_max1yr=subx_smooth[int(subx.shape[0]/2):int(subx.shape[0])].apply(np.max)
                    end_date1yr=subx_smooth[int(subx.shape[0]/2):int(subx.shape[0])].apply(lambda x: x.idxmax())
                    duration1yr=end_date1yr.dt.dayofyear.sub(onset_date1yr.dt.dayofyear)
                    #duration1yr=pd.DataFrame(end_date1yr.dt.dayofyear.values-onset_date1yr.dt.dayofyear.values)
                    #if(end_date-onset_date)...how to find minimum between two dates at each cell?
                    junk=[]
                    for col in range(subx_smooth.shape[1]):
                        junk.append(np.min(subx_smooth[col][onset_date1yr[col]:end_date1yr[col]]))
                    val_min1yr=pd.DataFrame(np.array(junk),index=onset_max.index)
                    avg_max1yr=end_max1yr.add(onset_max1yr)
                    avg_max1yr=avg_max1yr/2.0
                    #intensity1yr=avg_max1yr.sub(val_min1yr)
                    #intensity1yr=pd.DataFrame(((end_max1yr.values+onset_max1yr.values)/2.0)-val_min1yr.values)
                
                    #add current year's data to output dataframe for period
                    avg_max=pd.concat([avg_max,avg_max1yr],axis=1,join_axes=[avg_max.index])
                    onset_max=pd.concat([onset_max,onset_max1yr],axis=1,join_axes=[onset_max.index])
                    onset_day=pd.concat([onset_day,onset_date1yr.dt.dayofyear],axis=1,join_axes=[onset_day.index])
                    end_max=pd.concat([end_max,end_max1yr],axis=1,join_axes=[end_max.index])
                    end_day=pd.concat([end_day,end_date1yr.dt.dayofyear],axis=1,join_axes=[end_day.index])
                    val_min=pd.concat([val_min,val_min1yr],axis=1,join_axes=[val_min.index])
                    duration=pd.concat([duration,duration1yr],axis=1,join_axes=[duration.index])
                    #intensity=pd.concat([intensity,intensity1yr],axis=1,join_axes=[intensity.index])

                intensity=avg_max.sub(val_min)
                #print output statistics
                msd_stats['ID']=np.array(ids, dtype=int)
                msd_stats['Lon']=np.array(lons)
                msd_stats['Lat']=np.array(lats)
                msd_stats['nyears']=onset_max.count(axis=1)
                msd_stats['onset_mx_mn']=onset_max.mean(axis=1)
                msd_stats['onset_dy_mn']=onset_day.mean(axis=1)
                msd_stats['end_mx_mn']=end_max.mean(axis=1)
                msd_stats['end_dy_mn']=end_day.mean(axis=1)
                msd_stats['min_mn_mn']=val_min.mean(axis=1)
                msd_stats['duration_mn']=duration.mean(axis=1)
                msd_stats['intensity_mn']=intensity.mean(axis=1)
                msd_stats['onset_mx_sd']=onset_max.std(axis=1)
                msd_stats['onset_dy_sd']=onset_day.std(axis=1)
                msd_stats['end_mx_sd']=end_max.std(axis=1)
                msd_stats['end_dy_sd']=end_day.std(axis=1)
                msd_stats['min_mn_sd']=val_min.std(axis=1)
                msd_stats['duration_sd']=duration.std(axis=1)
                msd_stats['intensity_sd']=intensity.std(axis=1)
                foutname = outdir + "/" + gcm + "_" + scen + "_"+v+"_"+str(start_yr)+"-"+str(ey)+"_stats.csv"
                print "writing output to file " + foutname
                try:
                    os.remove(foutname)
                except OSError:
                    pass
                msd_stats.to_csv(foutname, index=False, float_format='%.4f')
